In [ ]:
import pandas as pd
test_df = pd.read_csv('../ag_news_csv/test.csv')
train_df = pd.read_csv('../ag_news_csv/train.csv')

In [4]:
train_sup_dict = {'label': y_train, 'sentence': train_text}
test_dict = {'label': y_test, 'sentence': test_text}
unsup_dict = {'sentence': unsup_text}


In [5]:
import pandas as pd
train_sup_df = pd.DataFrame.from_dict(train_sup_dict)
test_df = pd.DataFrame.from_dict(test_dict)
unsup_df = pd.DataFrame.from_dict(unsup_dict)


In [8]:
unsup_df.head


<bound method NDFrame.head of                                                 sentence
0      This movie is hilarious! It is just so funny. ...
1      Firstly, this is a film that is really about m...
2      Is the movie in ENGLISH? I have seen a lot of ...
3      So here we are watching home movies? No. No on...
4      James Caan must be wondering why he agreed to ...
...                                                  ...
49995  After THIS GUN FOR HIRE, Paramount realized th...
49996  I've seen many of the movies made about the li...
49997  I thought this movie was very good -- thought ...
49998  The song was awesome. Bobbie Gentry song is cl...
49999  One of the best in adult anime, this one has i...

[50000 rows x 1 columns]>

In [11]:
train_sup_df.to_csv('train_sup.csv', index=False)
unsup_df.to_csv('unsup.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [17]:
# 以上仅仅完成了将数据集处理成csv文件，接下来按照模型要求处理数据集
# 模型要求：70000个无监督数据，20/500/2500个训练数据，250000个测试数据

# 对于测试数据，保持不变，对于训练数据，分为两部分，一部分是无监督数据，一部分是有监督数据
# 从训练数据中，采样2500个训练数据，并下采样至500个，再下采样至20个，最后将剩余的和无监督混合

from sklearn.utils import shuffle
path = "./model_require/"

folder = os.path.exists(path)
if not folder:
    os.makedirs(path)

train = pd.read_csv('train_sup.csv')
# 不使用sample而是用shuffle是因为，这样可以便于将无监督数据分离出来
label0, label1 = shuffle(train[train['label'].isin([0])]), shuffle(
    train[train['label'].isin([1])])
train0, unsup0 = label0[:1250], label0[1250:]  # isin仅可提供一个list，所以0需要用list来表示
train1, unsup1 = label1[:1250], label1[1250:]
shuffle(pd.concat([train0, train1], ignore_index=True)
        ).to_csv(path+'train_2500.csv', index=False)
shuffle(pd.concat([train0.sample(250), train1.sample(250)], ignore_index=True)
        ).to_csv(path+'train_500.csv', index=False)
shuffle(pd.concat([train0.sample(10), train1.sample(10)], ignore_index=True)
        ).to_csv(path+'train_20.csv', index=False)


' train_2500 = shuffle(pd.concat([train0, train1]))\ntrain_500 = shuffle(pd.concat(train0.sample(250), train1.sample(250)))\ntrain_20 = shuffle(pd.concat(train0.sample(10), train1.sample(10))) '

In [49]:
unsup = pd.read_csv('unsup.csv')
unsup = pd.concat([unsup, pd.DataFrame(pd.concat(
    [unsup0[:10000], unsup1[:10000]], ignore_index=True), columns=['sentence'])], ignore_index=True)
shuffle(unsup).to_csv(path+'unsup.csv', index=False)

In [50]:
test=pd.read_csv('test.csv')
shuffle(test).to_csv(path+'test.csv', index=False)# 对测试数据进行打乱